# 05.03 - Pivot y Reshape

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio  
**Tiempo estimado:** 25 min

---

## Que aprenderemos?

- Diferencia entre formato wide y long
- Crear pivot tables
- Transformar con `melt()` (wide -> long)
- Transformar con `pivot()` (long -> wide)
- `stack()` y `unstack()`

In [1]:
import pandas as pd
import numpy as np

---

## 1. Formato Wide vs Long

**Wide:** Una columna por variable/periodo  
**Long:** Una fila por observacion (tidy data)

In [2]:
# Formato WIDE: columnas por mes
trips_wide = pd.DataFrame({
    "station": ["Sol", "Atocha", "Retiro"],
    "jan": [120, 85, 95],
    "feb": [135, 90, 100],
    "mar": [150, 110, 120]
})

print("Formato WIDE:")
trips_wide

Formato WIDE:


,station,jan,feb,mar
0,Sol,120,135,150
1,Atocha,85,90,110
2,Retiro,95,100,120


In [3]:
# Formato LONG: una fila por observacion
trips_long = pd.DataFrame({
    "station": ["Sol", "Sol", "Sol", "Atocha", "Atocha", "Atocha", "Retiro", "Retiro", "Retiro"],
    "month": ["jan", "feb", "mar", "jan", "feb", "mar", "jan", "feb", "mar"],
    "trips": [120, 135, 150, 85, 90, 110, 95, 100, 120]
})

print("Formato LONG:")
trips_long

Formato LONG:


,station,month,trips
0,Sol,jan,120
1,Sol,feb,135
2,Sol,mar,150
3,Atocha,jan,85
4,Atocha,feb,90
5,Atocha,mar,110
6,Retiro,jan,95
7,Retiro,feb,100
8,Retiro,mar,120


**Long es mejor para:**
- Analisis con pandas/SQL
- Visualizacion con seaborn
- Facilita groupby y agregaciones

---

## 2. `melt()`: Wide -> Long

In [4]:
print("Original (wide):")
display(trips_wide)

Original (wide):


,station,jan,feb,mar
0,Sol,120,135,150
1,Atocha,85,90,110
2,Retiro,95,100,120


In [5]:
# Convertir a long
melted = pd.melt(
    trips_wide,
    id_vars=["station"],       # Columnas a mantener
    value_vars=["jan", "feb", "mar"],  # Columnas a "derretir"
    var_name="month",          # Nombre para la nueva columna de variables
    value_name="trips"         # Nombre para la nueva columna de valores
)

print("Resultado (long):")
melted

Resultado (long):


,station,month,trips
0,Sol,jan,120
1,Atocha,jan,85
2,Retiro,jan,95
3,Sol,feb,135
4,Atocha,feb,90
5,Retiro,feb,100
6,Sol,mar,150
7,Atocha,mar,110
8,Retiro,mar,120


In [6]:
# Ordenar para mejor visualizacion
melted.sort_values(["station", "month"]).reset_index(drop=True)

,station,month,trips
0,Atocha,feb,90
1,Atocha,jan,85
2,Atocha,mar,110
3,Retiro,feb,100
4,Retiro,jan,95
5,Retiro,mar,120
6,Sol,feb,135
7,Sol,jan,120
8,Sol,mar,150


---

## 3. `pivot()`: Long -> Wide

In [7]:
print("Original (long):")
display(trips_long)

Original (long):


,station,month,trips
0,Sol,jan,120
1,Sol,feb,135
2,Sol,mar,150
3,Atocha,jan,85
4,Atocha,feb,90
5,Atocha,mar,110
6,Retiro,jan,95
7,Retiro,feb,100
8,Retiro,mar,120


In [8]:
# Convertir a wide
pivoted = trips_long.pivot(
    index="station",    # Filas
    columns="month",    # Columnas
    values="trips"      # Valores
)

print("Resultado (wide):")
pivoted

Resultado (wide):


month,feb,jan,mar
station,,,
Atocha,90,85,110
Retiro,100,95,120
Sol,135,120,150


In [9]:
# Limpiar el resultado
pivoted = pivoted.reset_index()
pivoted.columns.name = None  # Eliminar nombre de columnas
pivoted

,station,feb,jan,mar
0,Atocha,90,85,110
1,Retiro,100,95,120
2,Sol,135,120,150


---

## 4. `pivot_table()`: Pivot con agregacion

Cuando hay duplicados, necesitamos agregar.

In [10]:
# Datos con multiples viajes por estacion/mes
trips_detail = pd.DataFrame({
    "station": ["Sol", "Sol", "Sol", "Atocha", "Atocha", "Sol"],
    "month": ["jan", "jan", "feb", "jan", "feb", "jan"],
    "duration": [15, 22, 18, 30, 25, 12],
    "user_type": ["subscriber", "casual", "subscriber", "subscriber", "casual", "casual"]
})

trips_detail

,station,month,duration,user_type
0,Sol,jan,15,subscriber
1,Sol,jan,22,casual
2,Sol,feb,18,subscriber
3,Atocha,jan,30,subscriber
4,Atocha,feb,25,casual
5,Sol,jan,12,casual


In [11]:
# pivot_table agrega automaticamente
pivot = pd.pivot_table(
    trips_detail,
    values="duration",
    index="station",
    columns="month",
    aggfunc="mean"  # Funcion de agregacion
)

pivot

month,feb,jan
station,,
Atocha,25.0,30.000000
Sol,18.0,16.333333


In [12]:
# Con conteo
pd.pivot_table(
    trips_detail,
    values="duration",
    index="station",
    columns="month",
    aggfunc="count"
)

month,feb,jan
station,,
Atocha,1,1
Sol,1,3


In [13]:
# Multiples agregaciones
pd.pivot_table(
    trips_detail,
    values="duration",
    index="station",
    columns="month",
    aggfunc=["mean", "count", "sum"]
)

mean            count     sum    
month     feb        jan   feb jan feb jan
station                                   
Atocha   25.0  30.000000     1   1  25  30
Sol      18.0  16.333333     1   3  18  49

### Pivot con multiples indices

In [14]:
pd.pivot_table(
    trips_detail,
    values="duration",
    index=["station", "user_type"],
    columns="month",
    aggfunc="mean",
    fill_value=0  # Rellenar NaN
)

month                feb   jan
station user_type             
Atocha  casual      25.0   0.0
        subscriber   0.0  30.0
Sol     casual       0.0  17.0
        subscriber  18.0  15.0

### Totales con margins

In [15]:
pd.pivot_table(
    trips_detail,
    values="duration",
    index="station",
    columns="month",
    aggfunc="sum",
    margins=True,
    margins_name="Total"
)

month,feb,jan,Total
station,,,
Atocha,25,30,55
Sol,18,49,67
Total,43,79,122


---

## 5. `stack()` y `unstack()`

Para trabajar con MultiIndex.

In [16]:
# Crear DataFrame con MultiIndex
df_multi = pd.pivot_table(
    trips_detail,
    values="duration",
    index="station",
    columns="month",
    aggfunc="mean"
)

print("Original:")
df_multi

Original:


month,feb,jan
station,,
Atocha,25.0,30.000000
Sol,18.0,16.333333


In [17]:
# stack(): columnas -> filas
stacked = df_multi.stack()
print("Stacked:")
stacked

Stacked:


station  month
Atocha   feb      25.000000
         jan      30.000000
Sol      feb      18.000000
         jan      16.333333
dtype: float64

In [18]:
# unstack(): filas -> columnas
unstacked = stacked.unstack()
print("Unstacked (vuelve al original):")
unstacked

Unstacked (vuelve al original):


month,feb,jan
station,,
Atocha,25.0,30.000000
Sol,18.0,16.333333


In [19]:
# unstack nivel diferente
stacked.unstack(level=0)  # Desapilar el primer nivel (station)

station,Atocha,Sol
month,,
feb,25.0,18.000000
jan,30.0,16.333333


---

## 6. Ejemplo practico: analisis mensual

In [20]:
# Datos de viajes mensuales
monthly_trips = pd.DataFrame({
    "station": ["Sol"]*6 + ["Atocha"]*6,
    "month": ["jan", "feb", "mar", "apr", "may", "jun"]*2,
    "trips": [120, 135, 150, 145, 160, 180, 85, 90, 110, 105, 115, 130],
    "revenue": [2400, 2700, 3000, 2900, 3200, 3600, 1700, 1800, 2200, 2100, 2300, 2600]
})

monthly_trips

,station,month,trips,revenue
0,Sol,jan,120,2400
1,Sol,feb,135,2700
2,Sol,mar,150,3000
3,Sol,apr,145,2900
4,Sol,may,160,3200
5,Sol,jun,180,3600
6,Atocha,jan,85,1700
7,Atocha,feb,90,1800
8,Atocha,mar,110,2200
9,Atocha,apr,105,2100


In [21]:
# Pivot para comparar estaciones
comparison = monthly_trips.pivot(
    index="month",
    columns="station",
    values="trips"
)

comparison

station,Atocha,Sol
month,,
apr,105,145
feb,90,135
jan,85,120
jun,130,180
mar,110,150
may,115,160


In [22]:
# Calcular diferencia
comparison["diff"] = comparison["Sol"] - comparison["Atocha"]
comparison

station,Atocha,Sol,diff
month,,,
apr,105,145,40
feb,90,135,45
jan,85,120,35
jun,130,180,50
mar,110,150,40
may,115,160,45


In [23]:
# Pivot table con ambas metricas
summary = pd.pivot_table(
    monthly_trips,
    values=["trips", "revenue"],
    index="station",
    aggfunc={"trips": "sum", "revenue": "sum"}
)

summary["avg_revenue_per_trip"] = summary["revenue"] / summary["trips"]
summary

,revenue,trips,avg_revenue_per_trip
station,,,
Atocha,12700,635,20.0
Sol,17800,890,20.0


---

## Resumen

| Funcion | Transforma | Uso |
|---------|------------|-----|
| `melt()` | Wide -> Long | Preparar para analisis |
| `pivot()` | Long -> Wide | Sin duplicados |
| `pivot_table()` | Long -> Wide | Con agregacion |
| `stack()` | Columnas -> Filas | MultiIndex |
| `unstack()` | Filas -> Columnas | MultiIndex |

---

**Anterior:** [05.02 - Merge y Join](05_02_merge_join.ipynb)  
**Siguiente:** [06.01 - Limpieza de Datos](../06_data_cleaning/06_01_missing_values.ipynb)